<a href="https://colab.research.google.com/github/kostas-panagiotakis/NLP/blob/main/Poetry_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BarbenHeimer with LLaMA 2


![image](https://images.lumacdn.com/cdn-cgi/image/format=auto,fit=cover,dpr=1,quality=75,width=960,height=480/event-covers/87/1f6d4850-0231-4bd9-92e8-af6b45c18d7a)


In the following notebook we'll be discussing Retrieval Augmented Generation - and how to leverage Meta's neweset LLM, LLaMA 2 as the engine!

### Pre-task Work

All we really need to do to get started is to get our prerequisites!

We'll be leveraging `langchain` and `llama 2` today.

Check out the docs:
- [LangChain](https://docs.langchain.com/docs/)
- [LLaMA 2](https://huggingface.co/blog/llama2)

In [ ]:
!pip install -U -q "langchain" "transformers==4.31.0" "datasets==2.13.0" "peft==0.4.0" "accelerate==0.21.0" "bitsandbytes==0.40.2" "trl==0.4.7" "safetensors>=0.3.1"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.7/973.7 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 45.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

### Task 1: Data Preparation

In this task we'll be collecting, and then parsing, our data.

#### Data Parsing

Now that we have our data - let's go ahead and start parsing it into a more usable format for LangChain!

We'll be using the `CSVLoader` for this application.

Check out the docs here:
- [CSVLoader](https://python.langchain.com/docs/integrations/document_loaders/csv)

In [ ]:
!pip install --upgrade langchain-community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.5 MB/s eta 0:00:00


In [36]:
from langchain.document_loaders.csv_loader import CSVLoader
import pandas as pd
from datasets import load_dataset

In [37]:
 # Load the dataset
dataset = load_dataset("isaacrehg/poetry-summary")
dataset
ds = dataset['train']

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/3420 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/isaacrehg___parquet/isaacrehg--poetry-summary-3700a5db81b0b26e/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [38]:
# Convert the dataset into df format
df = pd.DataFrame(ds)

source_col = "_id"

# Convert df to csv
poetry_data = df.to_csv('/content/output.csv', index=False)
file_path = '/content/output.csv'

poetry_data = CSVLoader(file_path=file_path, source_column=source_col)

poetry_data_loaded = poetry_data.load()


In [39]:
poetry_data_loaded[10]

Document(page_content='_id: 11\ntitle: [a] talisman\nauthor: marianne moore\nurl: https://poemanalysis.com/marianne-moore/talisman/\nsummary: ‘Talisman’ by Marianne Moore\xa0is a short, complex poem that speaks on a mysterious shipwreck the strange object found underneath it.\xa0\n-----\nThe poem describes a grounded ship with its mast torn from its hull, as well as the shepherd who stumbled upon it. Under its wreckage, the shepherd found a strange seagull shaped jewel, a talisman with an unknown purpose.\xa0\n-----\nYou can read the full poem here.', metadata={'source': '11', 'row': 10})

Now that we have collected our review information into a loader - we can go ahead and chunk the reviews into more manageable pieces.

We'll be leveraging the `RecursiveCharacterTextSplitter` for this task today.

While splitting our text seems like a simple enough task - getting this correct/incorrect can have massive downstream impacts on your application's performance.

You can read the docs here:
- [RecursiveCharacterTextSplitter](https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/recursive_text_splitter)

> ### HINT:
>It's always worth it to check out the LangChain source code if you're ever in a bind - for instance, if you want to know how to transform a set of documents, check it out [here](https://github.com/langchain-ai/langchain/blob/5e9687a196410e9f41ebcd11eb3f2ca13925545b/libs/langchain/langchain/text_splitter.py#L268C18-L268C18)

In [40]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000, # the character length of the chunk
    chunk_overlap = 100, # the character length of the overlap between chunks
    length_function = len, # the length function - in this case, character length (aka the python len() fn.)
)

In [41]:
poetry_documents = text_splitter.transform_documents(poetry_data_loaded)

In [42]:
len(poetry_documents)

4028

With our documents transformed into more manageable sizes, and with the correct metadata set-up, we're now ready to move on to creating our VectorStore!

### Task 2: Creating an "Index"

The term "index" is used largely to mean: Structured documents parsed into a useful format for querying, retrieving, and use in the LLM application stack.

#### Selecting Our VectorStore

There are a number of different VectorStores, and a number of different strengths and weaknesses to each.

In this notebook, we will be keeping it very simple by leveraging [Facebook AI Similarity Search](https://ai.meta.com/tools/faiss/#:~:text=FAISS%20(Facebook%20AI%20Similarity%20Search,more%20scalable%20similarity%20search%20functions.), or `FAISS`.

In [ ]:
!pip install -q -U faiss-cpu tiktoken sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 70.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 74.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 58.6 MB/s eta 0:00:00


We're going to be setting up our VectorStore with the OpenAI embeddings model. While this embeddings model does not need to be consistent with the LLM selection, it does need to be consistent between embedding our index and embedding our queries over that index.

While we don't have to worry too much about that in this example - it's something to keep in mind for more complex applications.

We're going to leverage a [`CacheBackedEmbeddings`](https://python.langchain.com/docs/modules/data_connection/caching_embeddings )flow to prevent us from re-embedding similar queries over and over again.

Not only will this save time, it will also save us precious embedding tokens, which will reduce the overall cost for our application.

>#### Note:
>The overall cost savings needs to be compared against the additional cost of storing the cached embeddings for a true cost/benefit analysis. If your users are submitting the same queries often, though, this pattern can be a massive reduction in cost.

In [43]:
from langchain.embeddings import CacheBackedEmbeddings, HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore

store = LocalFileStore("./cache/")

embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

core_embeddings_model = HuggingFaceEmbeddings(
    model_name=embed_model_id
)

embedder = CacheBackedEmbeddings.from_bytes_store(
    core_embeddings_model, store, namespace=embed_model_id
)

vector_store = FAISS.from_documents(poetry_documents, embedder)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Now that we've created the VectorStore, we can check that it's working by embedding a query and retrieving passages from our reviews that are close to it.

In [44]:
query = "Who wrote had i not been awake?"
embedding_vector = core_embeddings_model.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 4)

for page in docs:
  print(page.page_content)

_id: 1
title: 'had i not been awake'
author: seamus heaney
url: https://poemanalysis.com/seamus-heaney/had-i-not-been-awake/
summary: In ‘Had I not been awake‘, Heaney brilliantly depicts the humming energy of life that surrounds us everyday, but we rarely have cause to notice. 
-----
The poem begins with an extension of the title, expressing the narrator’s gratitude for being awake as it meant he did not miss the ambiguous “it”. Much of the poem’s intrigue stems from what this thing may be. Heaney employs his typically evocative poetic eye to the natural world around the narrator, notably the wind and its sounds. As the poem continues, it becomes clear that the narrator draws strength and inspiration from the power of nature, which emboldens them to continue living. 
-----
You can read the full poem here.
_id: 3154
title: to sleep
author: john keats
url: https://poemanalysis.com/john-keats/to-sleep/
summary:
_id: 1158
title: i wake and feel the fell of dark, not day
author: gerard man

Let's see how much time the `CacheBackedEmbeddings` pattern saves us:

In [ ]:
%%timeit -n 1 -r 1
query = "I really wanted to enjoy this and I know that I am not the target audience but there were massive plot holes and no real flow."
embedding_vector = embedder.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 4)

12.8 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
%%timeit
query = "I really wanted to enjoy this and I know that I am not the target audience but there were massive plot holes and no real flow."
embedding_vector = embedder.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 4)

7.69 ms ± 1.57 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


As we can see, even over a significant number of runs - the cached query is significantly faster than the first instance of the query!

With that, we're ready to move onto Task 3!

### Task 3: Building a Retrieval Chain

In this task, we'll be making a Retrieval Chain which will allow us to ask semantic questions over our data.

This part is rather abstracted away from us in LangChain and so it seems very powerful.

Be sure to check the documentation, the source code, and other provided resources to build a deeper understanding of what's happening "under the hood"!

#### A Basic RetrievalQA Chain

We're going to leverage `return_source_documents=True` to ensure we have proper sources for our reviews - should the end user want to verify the reviews themselves.

Hallucinations [are](https://arxiv.org/abs/2202.03629) [a](https://arxiv.org/abs/2305.15852) [massive](https://arxiv.org/abs/2303.16104) [problem](https://arxiv.org/abs/2305.18248) in LLM applications.

Though it has been tenuously shown that using Retrieval Augmentation [reduces hallucination in conversations](https://arxiv.org/pdf/2104.07567.pdf), one sure fire way to ensure your model is not hallucinating in a non-transparent way is to provide sources with your responses. This way the end-user can verify the output.

#### Our LLM

In this notebook, we're going to leverage Meta's LLaMA 2!

Specifically, we'll be using: `meta-llama/Llama-2-13b-chat-hf`

That's right, a 13B parameter model that we're going to run on *less than* 15GB of GPU RAM.

More information on this model can be found [here](https://huggingface.co/meta-llama/Llama-2-13b-chat-hf)

In [ ]:
!pip install huggingface-hub -q

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

We will be leveraging Tim Dettmer's `bitsandbytes` as well as `accelerate` and `transformers` from Hugging Face to make our model as small as possible. The overall quality of the model is fairly well retained!

In [ ]:
import torch
import transformers

model_id = "meta-llama/Llama-2-13b-chat-hf"

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

model_config = transformers.AutoConfig.from_pretrained(
    model_id
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto'
)

model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/33.4k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

You are calling `save_pretrained` to a 4-bit converted model, but your `bitsandbytes` version doesn't support it. If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed.


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 5120)
    (layers): ModuleList(
      (0-39): 40 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (k_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (v_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (o_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (up_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (down_proj): Linear4bit(in_features=13824, out_features=5120, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): Lla

In [45]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id
)

Now we need to pack it into a `pipeline` for compatability with `langchain`!

In [46]:
generate_text = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    return_full_text=False,
    temperature=0.3,
    max_new_tokens=1000
)

In [47]:
from langchain.llms import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=generate_text)

Now we can set up our chain.

In [48]:
retriever = vector_store.as_retriever()

In [49]:
from langchain.chains import RetrievalQA
from langchain.callbacks import StdOutCallbackHandler

handler = StdOutCallbackHandler()

qa_with_sources_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    callbacks=[handler],
    return_source_documents=True
)

Now that it's set-up, let's test it out!

In [50]:
qa_with_sources_chain({"query" : "Who wrote the poem had i not been awake?"})



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'Who wrote the poem had i not been awake?',
 'result': ' The poem "Had I not been awake" was written by Seamus Heaney.',
 'source_documents': [Document(page_content="_id: 1\ntitle: 'had i not been awake'\nauthor: seamus heaney\nurl: https://poemanalysis.com/seamus-heaney/had-i-not-been-awake/\nsummary: In ‘Had I not been awake‘, Heaney brilliantly depicts the humming energy of life that surrounds us everyday, but we rarely have cause to notice. \n-----\nThe poem begins with an extension of the title, expressing the narrator’s gratitude for being awake as it meant he did not miss the ambiguous “it”. Much of the poem’s intrigue stems from what this thing may be. Heaney employs his typically evocative poetic eye to the natural world around the narrator, notably the wind and its sounds. As the poem continues, it becomes clear that the narrator draws strength and inspiration from the power of nature, which emboldens them to continue living. \n-----\nYou can read the full poem here

In [51]:
qa_with_sources_chain({"query" : "What does the poem had i not been awake talk about?"})



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'What does the poem had i not been awake talk about?',
 'result': ' The poem talks about the power of nature and how it can give us strength and inspiration to continue living.',
 'source_documents': [Document(page_content="_id: 1\ntitle: 'had i not been awake'\nauthor: seamus heaney\nurl: https://poemanalysis.com/seamus-heaney/had-i-not-been-awake/\nsummary: In ‘Had I not been awake‘, Heaney brilliantly depicts the humming energy of life that surrounds us everyday, but we rarely have cause to notice. \n-----\nThe poem begins with an extension of the title, expressing the narrator’s gratitude for being awake as it meant he did not miss the ambiguous “it”. Much of the poem’s intrigue stems from what this thing may be. Heaney employs his typically evocative poetic eye to the natural world around the narrator, notably the wind and its sounds. As the poem continues, it becomes clear that the narrator draws strength and inspiration from the power of nature, which emboldens them to

In [52]:
qa_with_sources_chain({"query" : "What style of writing is the poem i not been awake written in?"})



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'What style of writing is the poem i not been awake written in?',
 'result': ' The poem "Had I not been awake" by Seamus Heaney is written in a style that is reminiscent of the Romantic poets, such as John Keats and Margaret Atwood, with its focus on nature and the power of the natural world to inspire and embolden the narrator. The poem also shares similarities with the work of Theodore Roethke, with its use of imagery and symbolism to convey the narrator\'s emotions and experiences.',
 'source_documents': [Document(page_content="_id: 1\ntitle: 'had i not been awake'\nauthor: seamus heaney\nurl: https://poemanalysis.com/seamus-heaney/had-i-not-been-awake/\nsummary: In ‘Had I not been awake‘, Heaney brilliantly depicts the humming energy of life that surrounds us everyday, but we rarely have cause to notice. \n-----\nThe poem begins with an extension of the title, expressing the narrator’s gratitude for being awake as it meant he did not miss the ambiguous “it”. Much of the p